In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sentence_transformers import SentenceTransformer
from sklearn.svm import SVC

# Approach Explanation

Sentence-BERT (SBERT), a modification of the pretrained BERT network that use siamese and triplet network structures to derive semantically meaningful sentence embeddings that can be compared using cosine-similarity.

1. Documents are tokenized into sentences.
2. Sentence Embeddings is extracted from the Sentence transformer : paraphrase-MiniLM-L6-v2 model for each sentence in documents.<br>
    <b>Note:</b> Embdedding size is 384 for each sentence.
3. Sentence embeddings are averaged out to create document emedding
4. document embeddings along with the label is used for training SVM Model.

In [2]:
train_data = pd.read_csv('train_dataset_multiclass.csv',index_col=0)
print(train_data.shape,end="\n\n")
print(train_data['Issue'].value_counts())
train_data.head()

(5908, 2)

Problem with a purchase shown on your statement    1993
Incorrect information on your report               1191
Managing an account                                 935
Fees or interest                                    904
Other features, terms, or problems                  885
Name: Issue, dtype: int64


,Consumer complaint narrative,Issue
57519,The Citibank Credit Card company is a ridiculo...,Problem with a purchase shown on your statement
2790,Hi I went to a local bar on the evening of XX/...,Problem with a purchase shown on your statement
54277,I transferred XXXX ThankYou Points ( Citibank ...,"Other features, terms, or problems"
587,I was in XXXX XXXX and used my CITIBANK credit...,Problem with a purchase shown on your statement
12420,I clicked a promotion on citibank 's website t...,Managing an account


In [3]:
test_data = pd.read_csv('test_dataset_multiclass.csv',index_col=0)
print(test_data.shape,end="\n\n")
print(test_data['Issue'].value_counts())
test_data.head()

(2532, 2)

Problem with a purchase shown on your statement    854
Incorrect information on your report               510
Managing an account                                401
Fees or interest                                   388
Other features, terms, or problems                 379
Name: Issue, dtype: int64


,Consumer complaint narrative,Issue
17357,"Back in 2015, we noticed a charge on our Macy ...",Fees or interest
6524,"- On XX/XX/XXXX, I purchased 1 ticket XXXX {$3...",Problem with a purchase shown on your statement
23689,I hope all is well ... \nThe reason why Im wri...,Fees or interest
6960,Citibank had requested some XXXX documents ear...,Managing an account
7204,On XX/XX/2021 I authorized a payment of amount...,Managing an account


In [4]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [5]:
train_data['Embeddings'] = train_data['Consumer complaint narrative'].apply(lambda x: np.mean(model.encode(sent_tokenize(x)),axis=0))

In [7]:
le = LabelEncoder()
le.fit(train_data['Issue'])

LabelEncoder()

In [11]:
le.transform(train_data['Issue'])

array([4, 4, 3, ..., 1, 0, 2])

In [12]:
train_data['Label'] = le.transform(train_data['Issue'])
train_data.head(2)

,Consumer complaint narrative,Issue,Embeddings,Label
57519,The Citibank Credit Card company is a ridiculo...,Problem with a purchase shown on your statement,"[-0.2623762, -0.222208, -0.35714966, 0.0858370...",4
2790,Hi I went to a local bar on the evening of XX/...,Problem with a purchase shown on your statement,"[-0.09492934, 0.12373071, -0.19660375, -0.0297...",4


In [13]:
test_data['Embeddings'] = test_data['Consumer complaint narrative'].apply(lambda x: np.mean(model.encode(sent_tokenize(x)),axis=0))

In [14]:
test_data['Label'] = le.transform(test_data['Issue'])
test_data.head(2)

,Consumer complaint narrative,Issue,Embeddings,Label
17357,"Back in 2015, we noticed a charge on our Macy ...",Fees or interest,"[0.0076910784, -0.15155736, -0.025334315, -0.0...",0
6524,"- On XX/XX/XXXX, I purchased 1 ticket XXXX {$3...",Problem with a purchase shown on your statement,"[-0.27412552, -0.13065003, -0.103624605, 0.148...",4


In [15]:
X_train = train_data['Embeddings'].apply(pd.Series)
y_train = train_data['Label']

X_test = test_data['Embeddings'].apply(pd.Series)
y_test = test_data['Label']

In [16]:
clf = SVC()
clf.fit(X_train,y_train)

SVC()

In [17]:
predictions = clf.predict(X_test)

In [18]:
accuracy_score(y_test,predictions)

0.7883096366508688

In [19]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.74      0.76       388
           1       0.80      0.83      0.81       510
           2       0.81      0.85      0.83       401
           3       0.69      0.53      0.60       379
           4       0.80      0.87      0.84       854

    accuracy                           0.79      2532
   macro avg       0.78      0.76      0.77      2532
weighted avg       0.78      0.79      0.78      2532



In [ ]:
print(confus)